##### Import Libraries

In [7]:
import pandas as pd

##### Read datasets

In [8]:
CJ_releases_df = pd.read_csv('C:/MachineLearning/CJ_Releases_2022.csv')
CJ_receives_df = pd.read_csv('C:/MachineLearning/CJ_Receives_2022.csv')

### Phase I

In [4]:
# Column rename
CJ_releases_df.rename(columns={'Release Date': 'Date', 'Release Type': 'Type'}, inplace=True)
CJ_receives_df.rename(columns={'Receive Date': 'Date', 'Receive Type': 'Type'}, inplace=True)

CJ_releases_df['Record Type'] = 'Release'
CJ_receives_df['Record Type'] = 'Receive'


##### Datasets Integration

In [ ]:

CriminalJustice_df = pd.concat([CJ_releases_df, CJ_receives_df], ignore_index=True)

file_path = 'C:\\MachineLearning\\CriminalJustice_df.csv'
CriminalJustice_df.to_csv(file_path, index=False)